Leung Wai Liu <br>
JPMorgan AI Research <br>
August 8, 2022 <br>
Twitter Relabelling WORD Train Data Processing 

In [64]:
from reading_datasets import read_ud_dataset, reading_tb_ner
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

In [65]:
pos_train = read_ud_dataset(dataset = 'tb', location = '../Datasets/POSTagging/Tweebank/', split = 'train')
pos_val = read_ud_dataset(dataset = 'tb', location = '../Datasets/POSTagging/Tweebank/', split = 'dev')
pos_test = read_ud_dataset(dataset = 'tb', location = '../Datasets/POSTagging/Tweebank/', split = 'test')
ner_train = reading_tb_ner(location = '../Datasets/NER/Tweebank/', split = 'train')
ner_val = reading_tb_ner(location = '../Datasets/NER/Tweebank/', split = 'dev')
ner_test = reading_tb_ner(location = '../Datasets/NER/Tweebank/', split = 'test')

#Getting all the tweets for training dataset
combined_dataset_train = []
for i, (tweet, pos_labels) in enumerate(pos_train):
    tweet = [token.lower() for token in tweet]
    query = ' '.join(tweet)

    found = False
    for ner_tweet, ner_labels in ner_train:
        ner_tweet = [token.lower() for token in ner_tweet]
        check = ' '.join(ner_tweet)
        if query == check:
            found = True
            break

    if found and tweet == ner_tweet:
        combined_dataset_train.append([tweet, pos_labels, ner_labels])


In [66]:
combined_train_ner_per = []
combined_train_ner_org = []
combined_train_ner_loc = []
combined_train_ner_misc = []
combined_train_ner_o = []

for tweet, pos_labels, ner_labels in combined_dataset_train:
    for t, p, n in zip(tweet, pos_labels, ner_labels):
        if n == "O":
            combined_train_ner_o.append(t)
        else:
            new_n = n[2:]
            if new_n == "PER":
                combined_train_ner_per.append(t)
            elif new_n == "ORG":
                combined_train_ner_org.append(t)
            elif new_n == "LOC":
                combined_train_ner_loc.append(t)
            elif new_n == "MISC":
                combined_train_ner_misc.append(t)


In [67]:
train_ner_per_counter = Counter(combined_train_ner_per)

print(len(train_ner_per_counter))


406


In [68]:
per_counter_df = pd.DataFrame.from_dict(train_ner_per_counter, orient='index').reset_index()
per_counter_df = per_counter_df.rename(columns={'index': 'PER', 0: 'Count'})
per_counter_df = per_counter_df.sort_values(by="Count", ascending=False, ignore_index=True)
per_counter_df

,PER,Count
0,trump,8
1,god,7
2,lebron,6
3,drake,4
4,prince,4
...,...,...
401,gareth,1
402,gunsmithcat,1
403,prather,1
404,mrs.,1


In [69]:
train_ner_org_counter = Counter(combined_train_ner_org)

print(len(train_ner_org_counter))

283


In [70]:
org_counter_df = pd.DataFrame.from_dict(train_ner_org_counter, orient='index').reset_index()
org_counter_df = org_counter_df.rename(columns={'index': 'ORG', 0: 'Count'})
org_counter_df = org_counter_df.sort_values(by="Count", ascending=False, ignore_index=True)
org_counter_df

,ORG,Count
0,twitter,9
1,apple,7
2,facebook,6
3,snapchat,5
4,heat,4
...,...,...
278,smart,1
279,operanightclub,1
280,fort,1
281,wayne,1


In [71]:
train_ner_loc_counter = Counter(combined_train_ner_loc)

print(len(train_ner_loc_counter))

142


In [72]:
loc_counter_df = pd.DataFrame.from_dict(train_ner_loc_counter, orient='index').reset_index()
loc_counter_df = loc_counter_df.rename(columns={'index': 'LOC', 0: 'Count'})
loc_counter_df = loc_counter_df.sort_values(by="Count", ascending=False, ignore_index=True)
loc_counter_df

,LOC,Count
0,uk,3
1,chicago,3
2,usa,3
3,london,3
4,america,3
...,...,...
137,libya,1
138,ny,1
139,cuba,1
140,francisco,1


In [73]:
train_ner_misc_counter = Counter(combined_train_ner_misc)

print(len(train_ner_misc_counter))

376


In [74]:
misc_counter_df = pd.DataFrame.from_dict(train_ner_misc_counter, orient='index').reset_index()
misc_counter_df = misc_counter_df.rename(columns={'index': 'MISC', 0: 'Count'})
misc_counter_df = misc_counter_df.sort_values(by="Count", ascending=False, ignore_index=True)
misc_counter_df

,MISC,Count
0,the,19
1,of,6
2,i,6
3,-,4
4,day,3
...,...,...
371,tour,1
372,north,1
373,k-pop,1
374,serophene,1


In [75]:
train_merged_together = pd.concat([per_counter_df, org_counter_df, loc_counter_df, misc_counter_df], axis=1)
train_merged_together

,PER,Count,ORG,Count,LOC,Count,MISC,Count
0,trump,8,twitter,9.0,uk,3.0,the,19.0
1,god,7,apple,7.0,chicago,3.0,of,6.0
2,lebron,6,facebook,6.0,usa,3.0,i,6.0
3,drake,4,snapchat,5.0,london,3.0,-,4.0
4,prince,4,heat,4.0,america,3.0,day,3.0
...,...,...,...,...,...,...,...,...
401,gareth,1,NaN,NaN,NaN,NaN,NaN,NaN
402,gunsmithcat,1,NaN,NaN,NaN,NaN,NaN,NaN
403,prather,1,NaN,NaN,NaN,NaN,NaN,NaN
404,mrs.,1,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
train_merged_together.to_csv('logs/train_merged_together.tsv', sep='\t', index=False)